### Part II - VacationPy

Now let's use your skills in working with weather data to plan future vacations. Use jupyter-gmaps and the Google Places API for this part of the assignment.

* **Note:** Remember that any API usage beyond the $200 credit will be charged to your personal account. You can set quotas and limits to your daily requests to be sure you can't be charged. Check out [Google Maps Platform Billing](https://developers.google.com/maps/billing/gmp-billing#monitor-and-restrict-consumption) and [Manage your cost of use](https://developers.google.com/maps/documentation/javascript/usage-and-billing#set-caps) for more information.

* **Note:** if you having trouble displaying the maps, try running `jupyter nbextension enable --py gmaps` in your environment and retry.

To complete this part of the assignment,you will need to do the following:

* Create a heat map that displays the humidity for every city from Part I.

  ![heatmap](Images/heatmap.png)

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

  ![hotel map](Images/hotel_map.png)

As final considerations:

* You must complete your analysis using a Jupyter notebook.
* You must use the Matplotlib or Pandas plotting libraries.
* For Part I, you must include a written description of three observable trends based on the data.
* For Part II, you must include a screenshot of the heatmap you create and include it in your submission.
* You must use proper labeling of your plots, including aspects like: Plot Titles (with date of analysis) and Axes Labels.
* For max intensity in the heat map, try setting it to the highest humidity found in the data set.

# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [183]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [184]:
# load csv file

df_city_weather = pd.read_csv('output_data/city_weather_data.csv')

df_city_weather.head()

,city,lat,lon,temp_max,humidity,cloudiness,windspeed,country,date/time
0,vao,-22.6667,167.4833,74.43,72,92,26.46,NC,1619935944
1,cape town,-33.9258,18.4232,61.00,93,0,5.75,ZA,1619935944
2,nanortalik,60.1432,-45.2371,34.79,69,85,12.97,GL,1619935945
3,port blair,11.6667,92.7500,86.58,66,14,10.56,IN,1619935945
4,ayan,56.4500,138.1667,32.54,85,97,8.77,RU,1619935946


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [185]:
# create heat map using gmaps and city data

locations = df_city_weather[['lat', 'lon']]
weights = df_city_weather['humidity']
fig = gmaps.figure(layout={'width': '800px', 'height': '500px'},zoom_level=1.5, center = (30,0))
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='500px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [186]:
# drop rows that are missing any data
df_city_weather.dropna(axis=0, how='any', inplace=True)

# find cities with ideal weather conditions
df_city_weather_ideal = df_city_weather[(df_city_weather['temp_max'].between(65, 80, inclusive=True)) & (df_city_weather['cloudiness'] <= 40) & (df_city_weather['humidity'] <= 60)]

df_city_weather_ideal

,city,lat,lon,temp_max,humidity,cloudiness,windspeed,country,date/time
37,asyut,27.1810,31.1837,75.20,29,0,10.36,EG,1619935957
47,merrill,45.1805,-89.6835,73.40,35,1,8.05,US,1619935961
119,meadow lake,34.8014,-106.5436,71.60,15,20,4.61,US,1619935987
131,hervey bay,-25.2986,152.8535,75.20,60,40,16.11,AU,1619935992
145,tinos,37.5375,25.1634,77.00,44,20,6.91,GR,1619935997
220,matamoros,25.5333,-103.2500,66.99,37,0,1.77,MX,1619936024
230,geraldton,-28.7667,114.6000,75.20,31,0,20.71,AU,1619936028
254,chardara,41.2547,67.9692,74.46,49,0,1.63,KZ,1619936037
255,ixtapa,20.7000,-105.2000,65.52,57,0,2.93,MX,1619936037
293,saint george,37.1041,-113.5841,73.99,13,1,10.36,US,1619936050


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [187]:
# store lat, lon coordiates for ideal cities in variable

cities_coords = list(zip(list(df_city_weather_ideal['lat']), list(df_city_weather_ideal['lon'])))

cities_coords

[(27.180999999999997, 31.1837),
 (45.1805, -89.6835),
 (34.8014, -106.5436),
 (-25.2986, 152.8535),
 (37.5375, 25.1634),
 (25.5333, -103.25),
 (-28.7667, 114.6),
 (41.2547, 67.9692),
 (20.7, -105.2),
 (37.1041, -113.5841),
 (20.5022, -10.0711),
 (-35.1167, 147.3667),
 (20.6204, -105.2307),
 (41.1239, -100.7654),
 (28.7167, -111.5833),
 (41.8363, 59.9666),
 (36.5281, 37.9549),
 (39.7432, 98.5174),
 (25.7, 100.1833)]

In [188]:
# create a function that uses google places API and searches for nearby locations

def google_search_nearby(lat, lon, radius, place_type):
    
    '''
    This function uses the Google Places API to find the closest place to given latitude and longitude coordinates.
    * The arguments are: (1) latitude, (2) longitude, (3) search radius (in meters), and (4) place type
    * The function returns the nearest for the given arguments
    * The function returns None if no place is found with the given arguments
    
    Supported place types can be found here: https://developers.google.com/maps/documentation/places/web-service/supported_types
    The function requires an API key from Google cloud services: https://cloud.google.com/
    '''
    query_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?radius={radius}&type={place_type}&location={str(lat)},{str(lon)}&key={g_key}'
    
    response = requests.get(query_url)

    if response.status_code == 200:
        search_data = response.json()
        if len(search_data['results']) == 0:
            return None
        else:
            location_name = search_data['results'][0]['name']
            return location_name
    
    else:
        return None

In [189]:
radius = 5000
location_type = 'lodging'

hotel_list = [google_search_nearby(cities_coords[num][0], cities_coords[num][1], radius, location_type) for num in np.arange(0, len(cities_coords))]
hotel_list

['Florence Hotel Assiut',
 'Prairie Motel',
 'JKL Inc. R.V Park and Storage',
 'Shelly Bay Resort',
 'Hotel Poseidonio',
 'Quinta Belem',
 'Broadwater Mariner Resort',
 'Жанаргул',
 'Hotel Plaza Providencia',
 'Best Western Plus Abbey Inn',
 None,
 'The Club Motel and Apartments',
 'Sheraton Buganvilias Resort & Convention Center',
 'Holiday Inn Express & Suites North Platte, an IHG Hotel',
 None,
 'Hotel Erik',
 'Manbej City Hotel',
 'Jiuquan Hotel',
 'Dali Travelling With Hostel']

In [190]:
hotel_df = df_city_weather_ideal.assign(hotel=hotel_list)
hotel_df['city'] = hotel_df['city'].str.capitalize()
hotel_df

,city,lat,lon,temp_max,humidity,cloudiness,windspeed,country,date/time,hotel
37,Asyut,27.1810,31.1837,75.20,29,0,10.36,EG,1619935957,Florence Hotel Assiut
47,Merrill,45.1805,-89.6835,73.40,35,1,8.05,US,1619935961,Prairie Motel
119,Meadow lake,34.8014,-106.5436,71.60,15,20,4.61,US,1619935987,JKL Inc. R.V Park and Storage
131,Hervey bay,-25.2986,152.8535,75.20,60,40,16.11,AU,1619935992,Shelly Bay Resort
145,Tinos,37.5375,25.1634,77.00,44,20,6.91,GR,1619935997,Hotel Poseidonio
220,Matamoros,25.5333,-103.2500,66.99,37,0,1.77,MX,1619936024,Quinta Belem
230,Geraldton,-28.7667,114.6000,75.20,31,0,20.71,AU,1619936028,Broadwater Mariner Resort
254,Chardara,41.2547,67.9692,74.46,49,0,1.63,KZ,1619936037,Жанаргул
255,Ixtapa,20.7000,-105.2000,65.52,57,0,2.93,MX,1619936037,Hotel Plaza Providencia
293,Saint george,37.1041,-113.5841,73.99,13,1,10.36,US,1619936050,Best Western Plus Abbey Inn


In [191]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [193]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
locations = df_city_weather[['lat', 'lon']]
weights = df_city_weather['humidity']
fig = gmaps.figure(layout={'width': '800px', 'height': '500px'},zoom_level=1.5, center = (30,0))
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
marker_layer = gmaps.marker_layer(cities_coords, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='500px', width='800px'))